In [1]:
import open3d as o3d

import json
import numpy as np
from scipy.spatial.transform import Rotation as R
from compas.geometry import oriented_bounding_box_numpy
#from pytorch3d.ops import box3d_overlap
import torch
import os
import open3d as o3d

IOU_THRESHOLD = 0.2
LABEL_FOLDER = "./"

reg_json = [f for f in os.listdir(LABEL_FOLDER) if f.endswith('.json') and "no" not in f and "registration" in f][0]
not_reg_json = [f for f in os.listdir(LABEL_FOLDER) if f.endswith('.json') and "no" in f and "registration" in f][0]

reg_json = json.load(open(os.path.join(LABEL_FOLDER, reg_json)))
not_reg_json = json.load(open(os.path.join(LABEL_FOLDER, not_reg_json)))

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
points_unit = [
        [0, 0, 0],
        [1, 0, 0],
        [1, 1, 0],
        [0, 1, 0],
        [0, 0, 1],
        [1, 0, 1],
        [1, 1, 1],
        [0, 1, 1],
    ]
box = oriented_bounding_box_numpy(np.array(points_unit))
box

[[0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 1.0, 0.0],
 [0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0],
 [1.0, 0.0, 1.0],
 [1.0, 1.0, 1.0],
 [0.0, 1.0, 1.0]]

In [3]:
def reorder_vertices(box):
    """
    A unit cube, defined with the pythorch3d convention, has the following vertices:
        [0, 0, 0],                    (4) +---------+. (5)
        [1, 0, 0],                        | ` .     |  ` .
        [1, 1, 0],                        | (0) +---+-----+ (1)
        [0, 1, 0],                        |     |   |     |
        [0, 0, 1],                    (7) +-----+---+. (6)|
        [1, 0, 1],                        ` .   |     ` . |
        [1, 1, 1],                        (3) ` +---------+ (2)
        [0, 1, 1],
    
    While the OrientedBoundingBox from Open3D has the following vertices:
       [0, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [1, 1, 1],
       [0, 1, 1],
       [1, 0, 1],
       [1, 1, 0]

    This function reorders the vertices of the OrientedBoundingBox to match the pytorch3d convention.
    This allows to use the official IoU function from Pytorch3D.
    """
    assert box.shape == (8, 3)

    reordered = [
        box[0], box[1], box[7], box[2],
        box[3], box[6], box[4], box[5]
    ]
    return np.array(reordered)

In [35]:
for obj in reg_json['objects'][7:]:
    if obj['label'] == "chair":
        couch_reg = obj
        break

for obj in not_reg_json['objects'][7:]:
    if obj['label'] == "chair":
        couch_not_reg = obj
        break

couch_reg_pcd = o3d.geometry.PointCloud()
couch_not_reg_pcd = o3d.geometry.PointCloud()

couch_reg_pcd.points = o3d.utility.Vector3dVector(np.array(couch_reg['points']).reshape(-1, 3))
couch_not_reg_pcd.points = o3d.utility.Vector3dVector(np.array(couch_not_reg['points']).reshape(-1, 3))
couch_not_reg_pcd.translate(np.array([1.0, 1.0, 0]))


PointCloud with 953 points.

In [36]:
def create_mesh(pcd):
    radii = [0.005, 0.01, 0.02, 0.04, 0.08, 0.12, 0.16, 0.2, 0.25, 1]
    pcd.estimate_normals()
    rec_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
        pcd, o3d.utility.DoubleVector(radii))
    return rec_mesh

In [37]:
def custom_draw_geometry(reg, not_reg):
    vis = o3d.visualization.Visualizer()
    vis.create_window()

    #color pcd
    reg.paint_uniform_color([1, 0, 0])
    not_reg.paint_uniform_color([0, 1, 0])

    vis.add_geometry(reg)
    vis.add_geometry(not_reg)

    vis.run()
    vis.destroy_window()

In [38]:
custom_draw_geometry(couch_reg_pcd, couch_not_reg_pcd)

In [90]:
reg_mesh = create_mesh(couch_reg_pcd)
not_reg_mesh = create_mesh(couch_not_reg_pcd)
custom_draw_geometry(reg_mesh, not_reg_mesh)